# Evaluate TF-lite model to recognize sidewalks. 



In [0]:
!pip install tensorflow-gpu==2.0.0
!pip install tensorflow_hub

from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

### Initialize and load model

In [1]:
# Set location of test data and saved models on google drive
#data_path = "LA_test1_rev2"
data_path = "LA_train2"
#data_path = "040120"
#image_dir = 'drive/My Drive/CoLab_Data/Hollywood/'
image_dir = 'drive/My Drive/CoLab_Data/LosAngeles/'
saved_model_dir = 'drive/My Drive/CoLab_Data/save/'


# import dependencies
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import numpy as np
import matplotlib.pyplot as plt
%tensorflow_version 2.x
import tensorflow as tf
from google.colab import files
from google.colab import drive
from sklearn.metrics import classification_report, confusion_matrix

# connect to Google drive
drive.mount('/content/drive')

# unzip training images to local colab drive
unzip_path = '%s%s.zip' % (image_dir, data_path)
print(unzip_path)
os.system('unzip "%s%s.zip"' % (image_dir, data_path))
#os.system('rm %s/*.txt' % data_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive/My Drive/CoLab_Data/LosAngeles/LA_train2.zip


256

In [0]:
# Load tflite model directly 
model_name = 'mobileV2_LA_0407'
#model = tf.keras.models.load_model(saved_model_dir + model_name + '.tflite')
tflite_interpreter = tf.lite.Interpreter(model_path=saved_model_dir + model_name + '.tflite')

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])



In [2]:
# or load float model

# Load float model  
model_name = 'mobileV2_LA_0407'
model = tf.keras.models.load_model(saved_model_dir + model_name + '.h5')


In [0]:
# optionally load weights from checkpoint
checkpoint = 'cp-0015.ckpt'
checkpoint_path = 'drive/My Drive/CoLab_Data/save/mobileV2_LA_0407'
model.load_weights(checkpoint_path + '/' + checkpoint)

In [3]:
# Convert from float model to tflite model

TFLITE_MODEL = 'mobileV2_LA_0407a.tflite'

# Load float model  
model_name = 'mobileV2_LA_0407'
model = tf.keras.models.load_model(saved_model_dir + model_name + '.h5')


####### Convert for 2.0 using concrete functions #####
# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : model(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(tflite_model)

# load Tflite model
tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()



In [4]:

os.system('rm -r %s/val/side' % data_path)
os.system('rm -r %s/val/bike_lane' % data_path)
os.system('rm -r %s/val/street' % data_path)
tf.__version__

'2.0.0'

In [5]:
# run inference on images


# define image size and classes
IMAGE_SIZE = 224
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
CLASS_NAMES=('bike_lane', 'crosswalk', 'side','street')

BATCH_SIZE = 100

#eval_folder = "train"
eval_folder = "val"
data_path_eval = data_path + '/' + eval_folder
print(data_path_eval)

# open file for inference results
outFile = image_dir + data_path + eval_folder + '_out_data.txt'
outf = {'frame' : 0, 'label' : 1, 'class' : 2, 'swFlag' : 8, 'pos' : 9, 'vel' : 12}
outfile_format = '%5s,%d,' + 6 * '%3.2f,' + '%d,' + 3 * '%14.9f,' + 3 * '%5.2f,' + '\n'
#fid = open(outFile, "w")

# create image generator
datagen_val = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
gen = datagen_val.flow_from_directory(
    data_path_eval,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle = False)
num_images = gen.n

# allocate tensors
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (BATCH_SIZE, 224, 224, 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (BATCH_SIZE, 4))
tflite_interpreter.allocate_tensors()

# run inference on images and write results to file
ix=0
lb = []
y_pred = []
for n in range(num_images // BATCH_SIZE + 1):
  print(n * BATCH_SIZE)
  image_batch, label_batch = next(gen)
  label_batch = np.argmax(label_batch, axis=1)

  #Y_pred = model.predict(image_batch)
  #y_pred=np.concatenate((y_pred,np.argmax(Y_pred, axis=1)))
  #lb=np.concatenate((lb, label_batch))

  tflite_interpreter.set_tensor(input_details[0]['index'], image_batch)
  tflite_interpreter.invoke()
  tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
  print("Prediction results shape:", tflite_model_predictions.shape)

  #for n in range(len(label_batch)):
  #  fn = gen.filenames[n+ix]
  #  img_num = fn[-19:-13] + fn[-12:-10] + fn[-9:-4]
  #  fid.write(outfile_format % (img_num, y_pred[ix+n], Y_pred[n][0], Y_pred[n][1],
  #            Y_pred[n][2], Y_pred[n][3], 0, 0, 0, 0, 0, 0, 0, 0, 0))
  #ix += BATCH_SIZE

#fid.close()

# create Confusion Matrix
print('Confusion Matrix')
cm = confusion_matrix(lb, y_pred)
cm_norm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=3)
print(cm_norm)

# Calculate summary numbers for side and street
#print('False positive: %5.1f%%' % (100 * cm_norm[0,1]))
#print('False negative: %5.1f%%' % (100 * cm_norm[1,0]))

# Calculate summary numbers for side, street, and crosswalk
print('False positive: %5.2f%%' % (100 * (cm[0,1] + cm[0,2] + cm[3,1] + cm[3,2]) / np.sum(cm)))
print('False negative: %5.2f%%' % (100 * (cm[1,0] + cm[1,3] + cm[2,0] + cm[2,3]) / np.sum(cm)))



LA_train2/val
Found 441 images belonging to 2 classes.
0
Prediction results shape: (100, 4)
100
Prediction results shape: (100, 4)
200


KeyboardInterrupt: ignored

In [6]:
print(tflite_model_predictions[0:100,:
                               ])

[[6.61599353e-09 9.99968171e-01 4.31540457e-06 2.75521743e-05]
 [1.20515153e-06 9.91672099e-01 6.03592616e-06 8.32069758e-03]
 [1.27343299e-07 9.99576509e-01 3.56583296e-05 3.87718901e-04]
 [1.10948224e-06 9.97642815e-01 9.37783261e-05 2.26219464e-03]
 [1.37709762e-07 9.99139309e-01 6.22028951e-04 2.38610941e-04]
 [3.98477606e-09 9.99049962e-01 7.41084281e-04 2.08870217e-04]
 [7.32235276e-06 9.99875426e-01 6.28895975e-08 1.17118994e-04]
 [5.49942815e-05 9.99683976e-01 3.85968946e-07 2.60726782e-04]
 [1.82137010e-04 9.97379720e-01 1.71448082e-05 2.42093555e-03]
 [8.99236625e-07 9.99032378e-01 1.43438901e-05 9.52394970e-04]
 [7.90224632e-08 9.99993443e-01 1.40654507e-07 6.29709712e-06]
 [7.99858981e-07 9.99883056e-01 5.70705254e-07 1.15444724e-04]
 [2.58742748e-05 9.85231161e-01 3.43310559e-04 1.43997138e-02]
 [3.51282154e-07 9.98625875e-01 2.92303866e-05 1.34451897e-03]
 [1.02757243e-08 9.99996781e-01 1.15406380e-07 3.07558321e-06]
 [3.13575384e-07 9.99841094e-01 5.79053849e-05 1.006033

In [7]:
# Convert prediction results to Pandas dataframe, for better visualization
import pandas as pd

tflite_pred_dataframe = pd.DataFrame(tflite_model_predictions)
tflite_pred_dataframe.columns = ['bike_lane', 'crosswalk', 'sidewalk', 'street']

print("TFLite prediction results for the first elements")
tflite_pred_dataframe.head()

TFLite prediction results for the first elements


,bike_lane,crosswalk,sidewalk,street
0,6.615994e-09,0.999968,0.000004,0.000028
1,1.205152e-06,0.991672,0.000006,0.008321
2,1.273433e-07,0.999577,0.000036,0.000388
3,1.109482e-06,0.997643,0.000094,0.002262
4,1.377098e-07,0.999139,0.000622,0.000239


In [14]:
# Setup to plot incorrect images

batch_num = 0
BATCH_SIZE = 200
gen = datagen_val.flow_from_directory(
    data_path_eval,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle = False)

def show_batch(image_batch, label_batch):
  plt.figure(figsize=(40,40))
  plt_ix = 1
  for n in range(len(y_pred)):
      if label_batch[n] != y_pred[n] and plt_ix<=36:
        if label_batch[n] + y_pred[n] == 3:
          continue 
        ax = plt.subplot(6,6, plt_ix)
        plt_ix +=1
        plt.imshow(image_batch[n])
        plt.title(gen.filenames[batch_num * BATCH_SIZE + n] + '->' + CLASS_NAMES[y_pred[n]])
        plt.axis('off')

Found 23973 images belonging to 4 classes.


In [15]:
# plot incorrect images from next batch
image_batch, label_batch = next(gen)

label_batch = np.argmax(label_batch, axis=1)
Y_pred = model.predict(image_batch)
y_pred=np.argmax(Y_pred, axis=1)

show_batch(image_batch, label_batch)
batch_num += 1
print(batch_num)

1


<Figure size 2880x2880 with 0 Axes>

In [0]:
# download results
files.download(outFile)

In [0]:
 # Convert to TFlite, save and download results

model_name = 'mobileV2_LA_0407'

 # Save model 
model.save(model_name + '.h5')
model.save(saved_model_dir + model_name + '.h5')
tf.saved_model.save(model, model_name)

# convert to tflite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# save TFlite model to Google drive and download folder on local drive 
with open(saved_model_dir + model_name + '.tflite', 'wb') as f:
  f.write(tflite_model)
with open(model_name + '.tflite', 'wb') as f:
  f.write(tflite_model)
files.download(model_name + '.tflite')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: mobileV2_LA_0407/assets
